In [1]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# ##### 各関数での実行時間を保存するためのCSVの空データを作成するための処理 #####
# Processes = []
# for i in range(1,16):
#     Processes.append(i*i)
# for j in range(9):
#     Processes.append(2**j)
# Processes = list(set(Processes))
# Processes.sort()
# index = classes
# EmptyDF = pd.DataFrame(index=index, columns=Processes)
# for benchmark in benchmarks:
#     EmptyDF.to_csv(f"./csv_files/ExecTime@{benchmark}.csv")

##### 各ベンチマークの実行時間を記録する処理
# # Benchmarks = []
# # for benchmark in benchmarks:
# #     Benchmarks.append(benchmark.upper())
# # print(Benchmarks)

# Processes = [8, 16, 32, 64, 128, 256]
# for process in Processes:
#     TimeDataList = return_TimeDataList(process)
#     for DictData in TimeDataList:
#         benchmark = DictData.benchmark
#         process = DictData.process
#         BenchmarkClass = DictData.BenchmarkClass
#         time = DictData.time
#         FillCSV(benchmark=benchmark, process=process, BenchmarkClass=BenchmarkClass, time=time)

# # print(TimeDataList[0].return_AllData())


In [2]:
def GenerateMapeTable(FixedClass="B", FixedProcess="64", test_ratio=0.3):
    
    DirName = "./table_LatexForm/"
    FileSuffix = f"Train0{int(test_ratio*10)}.table"
    FilePrefix = "MapeTableFixed"
    
    input_MapeTableDF = return_MapeTableDataframe_FixedProcess(FixedProcess=FixedProcess, test_ratio=test_ratio)
    return_MapeTableDF = return_MapeTableOnlyStrDataframe(input_MapeTableDF=input_MapeTableDF)
    return_MapeTableDF.to_latex(f"{DirName}{FilePrefix}{FixedProcess}{FileSuffix}")
    input_MapeTableDF = return_MapeTableDataframe_FixedClass(FixedClass=FixedClass,test_ratio=test_ratio)
    return_MapeTableDF = return_MapeTableOnlyStrDataframe(input_MapeTableDF=input_MapeTableDF)
    return_MapeTableDF.to_latex(f"{DirName}{FilePrefix}{FixedClass}{FileSuffix}")

# MAPE表を生成する例   
# GenerateMapeTable()

In [13]:
benchmark = "cg"

def ConvertIndexNameToNumOfProfile(inputDF, Fixed="Process"):
    index = inputDF.index.tolist()
    # ConvertedIndexには入力データフレームの行名から使用されたプロファイルの数を格納している。
    ConvertedIndex = [0] * len(index)
    for i in range(len(index)):
        if(Fixed=="Process"):
            ConvertedIndex[i] = len(convert_StrListToStrList(index[i]))
        elif(Fixed=="Class"):
            ConvertedIndex[i] = len(convert_StrListToIntList(index[i]))
    returnDF = inputDF.copy(deep=True)
    returnDF.index = ConvertedIndex
    return(returnDF)

Fixed = "Process"
FixedProcessDF = return_ErrorRateFixedProcess_AveragePredictCostRealCost(benchmark=benchmark)
print(ConvertIndexNameToNumOfProfile(FixedProcessDF, Fixed=Fixed))

Fixed = "Class"
FixedClassDF = return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark=benchmark)
print(ConvertIndexNameToNumOfProfile(FixedClassDF, Fixed=Fixed))

     Average  PredictCosts  RealCost
4  0.0657727      16325.12  15784.96
3   0.135409        540.16  15784.96
1   0.288136         10.24  15784.96
     Average  PredictCosts  RealCost
9  0.0568636        913.82     166.4
8   0.102409        747.42     166.4
6   0.283636        537.50     166.4
4     5.4315        372.06     166.4
2    6.71836        210.46     166.4
